In [1]:
import os
import ctypes
from symfit import variables, Parameter, ODEModel, D, Fit, parameters, Variable
import numpy as np
from uncertainties import ufloat as uf
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from pathlib import Path as pt
from ipywidgets import widgets
from timescan import timescanplot
from time import time as start_time
os.getcwd()

'D:\\FELion_GUI3\\static\\assets\\python_files\\ODE'

In [2]:
%matplotlib inline

In [3]:
from widgetDefinitions import createWidgets

In [4]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [5]:
def definingParameter(value):
    exec(value)
    return locals()

def getValfromAddress(address):
    return ctypes.cast(address, ctypes.py_object).value

def getValfromVariable(variable):
    address = id(variable)
    return getValfromAddress(address)

In [6]:
import string
alphabets = string.ascii_uppercase

In [7]:
style = {'description_width': 'initial', 'width':"100"}
layout=widgets.Layout(width='50%')


In [23]:
twidget = createWidgets(filetype="scan", multiselect=False)
twidget.location.value = r"Z:\Students\Aravindh\Measurements\CD+\timescan"

In [24]:
# Z:\Students\Aravindh\Measurements\CD+\timescan

In [103]:
display(twidget.update_files_button)

time, mass = [], []

def init(location, filename):
    global time, mass, m
    file = pt(location) / filename
    time, mean, error, mass, t_res, t_b0  = timescanplot(file).get_data()
    m = timescanplot(file).get_fullmass()

    data = list(m.values())
    iplot(data)

out = widgets.interactive(init, location=twidget.location, filename=twidget.files)
display(out)

Button(button_style='success', description='update location', layout=Layout(width='20%'), style=ButtonStyle())

interactive(children=(Text(value='Z:\\Students\\Aravindh\\Measurements\\CD+\\timescan', description='scan loca…

In [105]:

totalReactionsNumber = len(mass)

numberOfReaction = widgets.BoundedIntText(description="numberOfReaction", style = style, value=totalReactionsNumber, layout=layout)
nameOfReactants = widgets.Text(description="nameOfReactants", style = style, value=", ".join([f"{i}u" for i in mass]), layout=layout)
labelnameOfReactants = widgets.Text(description="labelnameOfReactants", style = style, value="CD, CDHe, CDHe2, CDHe3, CDHe4, CDHe5", layout=layout)
nameOfParametersForward = widgets.Text(description="nameOfParametersForward", style = style,value=", ".join([f"k3{i}" for i in range(1, totalReactionsNumber)]), layout=layout)
nameOfParametersReverse = widgets.Text(description="nameOfParametersReverse", style = style,value=", ".join([f"CID{i}" for i in range(1, totalReactionsNumber)]), layout=layout)
parameterMinMaxStepForward = widgets.Text(description="parameterMinMaxStep", style = style, value="-40, -20, 0.001")
parameterMinMaxStepReverse = widgets.Text(description="parameterMinMaxStep", style = style, value="-30, -10, 0.001")

kineticsWidget = [numberOfReaction, nameOfReactants, labelnameOfReactants, nameOfParametersForward, parameterMinMaxStepForward, nameOfParametersReverse, parameterMinMaxStepReverse]

display(*kineticsWidget)

BoundedIntText(value=4, description='numberOfReaction', layout=Layout(width='50%'), style=DescriptionStyle(des…

Text(value='13.8u, 17.8u, 21.8u, 25.8u', description='nameOfReactants', layout=Layout(width='50%'), style=Desc…

Text(value='CD, CDHe, CDHe2, CDHe3, CDHe4, CDHe5', description='labelnameOfReactants', layout=Layout(width='50…

Text(value='k31, k32, k33', description='nameOfParametersForward', layout=Layout(width='50%'), style=Descripti…

Text(value='-40, -20, 0.001', description='parameterMinMaxStep', style=DescriptionStyle(description_width='ini…

Text(value='CID1, CID2, CID3', description='nameOfParametersReverse', layout=Layout(width='50%'), style=Descri…

Text(value='-30, -10, 0.001', description='parameterMinMaxStep', style=DescriptionStyle(description_width='ini…

In [89]:
timeStartIndexWidget = widgets.BoundedIntText(value=1, description="timeStartIndex",min=0, style=style)
endtimeIndexWidget = widgets.BoundedIntText(value=-1, description="endtimeIndex", min=-1, style=style)
display(timeStartIndexWidget, endtimeIndexWidget)

BoundedIntText(value=1, description='timeStartIndex', style=DescriptionStyle(description_width='initial'))

BoundedIntText(value=-1, description='endtimeIndex', min=-1, style=DescriptionStyle(description_width='initial…

In [90]:
timeStartIndex = timeStartIndexWidget.value
endtimeIndex = endtimeIndexWidget.value
print(f'{timeStartIndex=}, {endtimeIndex=}')

expTime = time[timeStartIndex:endtimeIndex] / 1000
fullData = {}

for key in nameOfReactants.value.split(", "):
    fullData[key] = m[key]['y'][timeStartIndex:endtimeIndex]
    
fullDataValues = np.array(list(fullData.values()))

timeStartIndex=1, endtimeIndex=-1


In [72]:
t = Variable("t")

reactantVariable = variables(labelnameOfReactants.value)
reactantVariableAddress = [id(i) for i in reactantVariable]

print(f"{reactantVariable=}\n{reactantVariableAddress=}")

label = labelnameOfReactants.value.split(", ")
if len(label) == 0:
    label = [alphabets[i] for i in range(len(totalReactionsNumber))]
    
initialCondition = fullDataValues.T[0]
print(initialCondition)

initialConditionWidgets = []


for i, val in enumerate(initialCondition):
    _widget = widgets.BoundedFloatText(value=val, description=f"{label[i]}+", max=1e20, min=0, step=20)
    initialConditionWidgets.append(_widget)
    
display(*initialConditionWidgets)

reactantVariable=(CD, CDHe)
reactantVariableAddress=[2103601148528, 2103586314016]
[1642.    6.]


BoundedFloatText(value=1642.0, description='CD+', max=1e+20, step=20.0)

BoundedFloatText(value=6.0, description='CDHe+', max=1e+20, step=20.0)

In [73]:
# Initial condition
initial_cond = {t:0}

for i, variable in enumerate(reactantVariable):
    initial_cond[getValfromVariable(variable)] = initialConditionWidgets[i].value
    
print(initial_cond)
list(initial_cond.keys())[1] == reactantVariable[0]

{t: 0, CD: 1642.0, CDHe: 6.0}


True

In [ ]:
tempWidget = widgets.BoundedFloatText(value=4.5, min=0, max=400, step=0.5)
pbeforeWidget = widgets.FloatLogSlider(base=10, value=1e-9, min=-10, max=-4)
paftereWidget = widgets.FloatLogSlider(base=10, value=1e-7, min=-10, max=-4)

In [119]:
# He/Ne number density
numberDensityValue = None
def numberDensity(temp, pbefore, pafter):
    global numberDensityValue
    constant = 4.18e17 # 1/boltzman_constant*sqrt(T)
    C = uf(205.54, 2.5) # Calibration factor 
    T = uf(temp, 0.1) # Temperature
    p = pafter - pbefore
    numberDensityValue = (constant*C*p)/(T**0.5)
    print(f"{pbefore=}, {pafter=}")
    print(numberDensityValue)
    return numberDensityValue.nominal_value

numberDensityWidgetOutput = widgets.interact(numberDensity, temp=tempWidget, pbefore=pbeforeWidget, pafter=paftereWidget)

interactive(children=(BoundedFloatText(value=4.5, description='temp', max=400.0, step=0.5), FloatLogSlider(val…

In [121]:
numberDensityValue

3540227760033.702+/-58322216346.722336

In [75]:
He = numberDensityValue.nominal_value
print(f"numberDensity: {He:.2e}")

rateConstantParametersForward = parameters(nameOfParametersForward.value)
rateConstantParametersReverse = parameters(nameOfParametersReverse.value)

rateEquation = {"forward":[He**2 * getValfromVariable(i) for i in rateConstantParametersForward],
                "reverse":[He * getValfromVariable(i) for i in rateConstantParametersReverse]}

rateEquation

numberDensity: 4.01e+12


{'forward': [1.60769308341119e+25*k31], 'reverse': [4009604822686.63*CID1]}

In [76]:
formationRate = {}

for i, forward, reverse in zip(range(len(rateEquation["forward"])), rateEquation["forward"], rateEquation["reverse"]):
    formationRate[f"{label[i]}"] = -getValfromVariable(forward)*getValfromVariable(reactantVariable[i]) + getValfromVariable(reverse)*getValfromVariable(reactantVariable[i+1])

formationRateList = list(formationRate.values())
formationRate[f"{label[-1]}"] =  -formationRateList[-1]
formationRateList = list(formationRate.values())
formationRate, reactantVariable

({'CD': 4009604822686.63*CID1*CDHe - 1.60769308341119e+25*k31*CD,
  'CDHe': -4009604822686.63*CID1*CDHe + 1.60769308341119e+25*k31*CD},
 (CD, CDHe))

In [77]:
rate_model = {D(getValfromVariable(reactantVariable[0]), t): getValfromVariable(formationRateList[0]) }
for i in range(1, totalReactionsNumber-1):
    rate_model[D(getValfromVariable(reactantVariable[i]), t)] =  getValfromVariable(formationRateList[i]) -getValfromVariable(formationRateList[i-1])
    
rate_model[D(getValfromVariable(reactantVariable[-1]), t)] = getValfromVariable(formationRateList[-1])

for i, j in rate_model.items():
    print(f"\n{i}:{j}")


Derivative(CD, t):4009604822686.63*CID1*CDHe - 1.60769308341119e+25*k31*CD

Derivative(CDHe, t):-4009604822686.63*CID1*CDHe + 1.60769308341119e+25*k31*CD


In [78]:
ode_model = ODEModel(rate_model, initial=initial_cond)

print(ode_model)

print(ode_model.dependent_vars[0] == list(ode_model.initial.keys())[1])

Derivative(CD, t; CID1, k31) = 4009604822686.63*CID1*CDHe - 1.60769308341119e+25*k31*CD
Derivative(CDHe, t; CID1, k31) = -4009604822686.63*CID1*CDHe + 1.60769308341119e+25*k31*CD
True


In [79]:
tdata = widgets.BoundedFloatText(
    value=5,
    min=1,
    max=20,
    step=0.25,
    description='Simulation:',
)
logPlot = widgets.Checkbox(description="Log", value=True)

In [80]:
def float_slider(_value, _description, _min = -40, _max = -20, steps = 1e-3):
    return widgets.FloatLogSlider(
                value=_value,
                base=10,
                min=_min,
                max=_max,
                step=steps,
                description=_description
            )

rateConstantSliderWidget = {"t":tdata}

_minF, _maxF, _stepF = parameterMinMaxStepForward.value.split(", ")
_minR, _maxR, _stepR = parameterMinMaxStepReverse.value.split(", ")

for forwarLabel, reverseLabel in zip(nameOfParametersForward.value.split(", "), nameOfParametersReverse.value.split(", ")):
    _sliderWidgetForward = float_slider(1e-30, forwarLabel, _minF, _maxF, _stepF)
    _sliderWidgetReverse = float_slider(1e-15, reverseLabel, _minR, _maxR, _stepR)
    
    rateConstantSliderWidget[forwarLabel] = _sliderWidgetForward
    rateConstantSliderWidget[reverseLabel] = _sliderWidgetReverse

rateConstantSliderWidget

{'t': BoundedFloatText(value=5.0, description='Simulation:', max=20.0, min=1.0, step=0.25),
 'k31': FloatLogSlider(value=1e-30, description='k31', max=-20.0, min=-40.0, step=0.001),
 'CID1': FloatLogSlider(value=1e-15, description='CID1', max=-10.0, min=-30.0, step=0.001)}

In [81]:
def func(log=True, **kwargs):
    
    print(kwargs)
    
    # Simulation time
    simulationTime = kwargs["t"] = np.linspace(0, kwargs["t"], 1000)
    
    simulationData = ode_model(**kwargs)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    color = 0
    for sim, exp, lg in zip(simulationData, fullDataValues, label):
        ax.plot(simulationTime, sim, f"C{color}", label=f"{lg}$^+$")
        ax.plot(expTime, exp, f".C{color}", ms=10)
        color += 1
        
    plt.ylabel('Counts')
    plt.xlabel('Time (s)')
    
    plt.grid()
    if log: plt.yscale('log')
    plt.legend(bbox_to_anchor=(1.2, 1))
    plt.show()
    plt.close('all')

out = widgets.interactive_output(func,  {"log":logPlot, **rateConstantSliderWidget})
display(*list(rateConstantSliderWidget.values()), logPlot, out)

BoundedFloatText(value=5.0, description='Simulation:', max=20.0, min=1.0, step=0.25)

FloatLogSlider(value=1e-30, description='k31', max=-20.0, min=-40.0, step=0.001)

FloatLogSlider(value=1e-15, description='CID1', max=-10.0, min=-30.0, step=0.001)

Checkbox(value=True, description='Log')

Output()

In [123]:
time.size, fullDataValues[0].size

(6, 4)

In [ ]:
ROSAA_kinetics_widgets = {"numberOfReaction":numberOfReaction, "nameOfReactants":nameOfReactants, "labelnameOfReactants":labelnameOfReactants, 
                          "nameOfParametersForward":nameOfParametersForward, "nameOfParametersReverse":nameOfParametersReverse, 
                          "parameterMinMaxStepForward":parameterMinMaxStepForward, "parameterMinMaxStepReverse":parameterMinMaxStepReverse,
                          "timeStartIndex":timeStartIndex, "endtimeIndex":endtimeIndex, "initialConditionValues":initialConditionValues
                         }

In [122]:
def ROSAA_kinetics(numberOfReaction, nameOfReactants, labelnameOfReactants, nameOfParametersForward, nameOfParametersReverse, parameterMinMaxStepForward, parameterMinMaxStepReverse
                  timeStartIndex, endtimeIndex, initialConditionValues, 
                  ):
    
    # Getting timescan data
    expTime = time[timeStartIndex:endtimeIndex] / 1000
    fullData = {}

    for key in nameOfReactants.split(", "):
        fullData[key] = m[key]['y'][timeStartIndex:endtimeIndex]

    fullDataValues = np.array(list(fullData.values()))
    
    # Making variables
    
    t = Variable("t")

    reactantVariable = variables(labelnameOfReactants)

    print(f"{reactantVariable=}\n{reactantVariableAddress=}")

    label = labelnameOfReactants.split(", ")
    
    if len(label) == 0:
        label = [alphabets[i] for i in range(len(totalReactionsNumber))]

    initialConditionValues = initialConditionValues.split(", ")

    # Initial condition
    initial_cond = {t:0}

    for i, variable in enumerate(reactantVariable):
        initial_cond[getValfromVariable(variable)] = initialConditionValues[i]

    print(f"{initial_cond=}")
    
    # Rate Equation
    He = numberDensityValue.nominal_value
    
    rateConstantParametersForward = parameters(nameOfParametersForward)
    rateConstantParametersReverse = parameters(nameOfParametersReverse)

    rateEquation = {"forward":[He**2 * getValfromVariable(i) for i in rateConstantParametersForward],
                    "reverse":[He * getValfromVariable(i) for i in rateConstantParametersReverse]}
    
    # Formation rate
    
    formationRate = {}

    for i, forward, reverse in zip(range(len(rateEquation["forward"])), rateEquation["forward"], rateEquation["reverse"]):
        formationRate[f"{label[i]}"] = -getValfromVariable(forward)*getValfromVariable(reactantVariable[i]) + getValfromVariable(reverse)*getValfromVariable(reactantVariable[i+1])

    formationRateList = list(formationRate.values())
    formationRate[f"{label[-1]}"] =  -formationRateList[-1]
    formationRateList = list(formationRate.values())
    
    # Rate Modal
    
    rate_model = {D(getValfromVariable(reactantVariable[0]), t): getValfromVariable(formationRateList[0])}
    
    for i in range(1, totalReactionsNumber-1):
        rate_model[D(getValfromVariable(reactantVariable[i]), t)] =  getValfromVariable(formationRateList[i]) -getValfromVariable(formationRateList[i-1])

    rate_model[D(getValfromVariable(reactantVariable[-1]), t)] = getValfromVariable(formationRateList[-1])

    for i, j in rate_model.items():
        print(f"\n{i}:{j}")
        

out = widgets.interactive_output(ROSAA_kinetics, )

SyntaxError: invalid syntax (<ipython-input-122-eb688561d306>, line 2)

In [68]:
#Fitting
t0 = start_time()

fit = Fit(ode_model, t=expTime, CD=fullDataValues[0], CDHe=fullDataValues[1])
fit_result = fit.execute()

t1 = start_time()
print(f'Time taken: {t1-t0} s')

print(fit_result)

Time taken: 0.5784268379211426 s

Parameter Value        Standard Deviation
CID1      1.000004e+00 1.308648e+13
k31       9.999996e-01 6.979428e+11
Status message         Desired error not necessarily achieved due to precision loss.
Number of iterations   1
Objective              <symfit.core.objectives.LeastSquares object at 0x000001E9C8E2F880>
Minimizer              <symfit.core.minimizers.BFGS object at 0x000001E9C8E2F7C0>

Goodness of fit qualifiers:
chi_squared            14330354.333330387
objective_value        7165177.166665194
r_squared              -25087.806056278194


In [63]:
fullDataValues

array([[1551.33333333, 1565.33333333, 1544.66666667],
       [  12.33333333,   29.33333333,   38.33333333]])

In [113]:
ode_model.independent_vars

[t]